<a href="https://colab.research.google.com/github/k2-fsa/colab/blob/master/sherpa-onnx/Run_distil_whisper_with_sherpa_onnx_on_GPU_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook shows how to run models from https://github.com/huggingface/distil-whisper with sherpa-onnx on GPU.

Please refer to
https://github.com/k2-fsa/colab/blob/master/sherpa-onnx/Run_distil_whisper_with_sherpa_onnx_on_CPU.ipynb
for how to run it on CPU.

In [1]:
import torch
a = torch.rand(100).cuda()
print(a.device)

cuda:0


# Build sherpa-onnx with GPU support

In [3]:
%%shell

git clone https://github.com/k2-fsa/sherpa-onnx
cd sherpa-onnx
mkdir build
cd build
cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_SHARED_LIBS=ON -DSHERPA_ONNX_ENABLE_GPU=ON ..
make -j

Cloning into 'sherpa-onnx'...
remote: Enumerating objects: 4530, done.
remote: Counting objects: 100% (1772/1772), done.
remote: Compressing objects: 100% (607/607), done.
remote: Total 4530 (delta 1374), reused 1242 (delta 1152), pack-reused 2758
Receiving objects: 100% (4530/4530), 2.42 MiB | 5.02 MiB/s, done.
Resolving deltas: 100% (2797/2797), done.
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning at CMakeLists.txt:63 (message):
  Compiling for NVIDIA GPU is enabled.  Please make sure cudatoolkit

  is installed on your 

# Download model files

In [4]:
%%shell
wget -q https://huggingface.co/csukuangfj/sherpa-onnx-whisper-distil-medium.en/resolve/main/distil-medium.en-decoder.int8.onnx
wget -q https://huggingface.co/csukuangfj/sherpa-onnx-whisper-distil-medium.en/resolve/main/distil-medium.en-encoder.int8.onnx
wget -q https://huggingface.co/csukuangfj/sherpa-onnx-whisper-distil-medium.en/resolve/main/distil-medium.en-tokens.txt
wget -q https://huggingface.co/csukuangfj/sherpa-onnx-whisper-distil-medium.en/resolve/main/test_wavs/0.wav
wget -q https://huggingface.co/csukuangfj/sherpa-onnx-whisper-distil-medium.en/resolve/main/test_wavs/1.wav
wget -q https://huggingface.co/csukuangfj/sherpa-onnx-whisper-distil-medium.en/resolve/main/test_wavs/8k.wav
ls -lh

total 548M
-rw-r--r--  1 root root 208K Nov  6 23:54 0.wav
-rw-r--r--  1 root root 523K Nov  6 23:54 1.wav
-rw-r--r--  1 root root  76K Nov  6 23:54 8k.wav
-rw-r--r--  1 root root 234M Nov  6 12:16 distil-medium.en-decoder.int8.onnx
-rw-r--r--  1 root root 313M Nov  6 12:32 distil-medium.en-encoder.int8.onnx
-rw-r--r--  1 root root 816K Nov  6 23:54 distil-medium.en-tokens.txt
drwxr-xr-x  1 root root 4.0K Nov  3 18:00 sample_data
drwxr-xr-x 21 root root 4.0K Nov  6 23:49 sherpa-onnx


# Run it with GPU

In [6]:
%%shell

export PATH=/content/sherpa-onnx/build/bin:$PATH

sherpa-onnx-offline \
  --whisper-encoder=./distil-medium.en-encoder.int8.onnx \
  --whisper-decoder=./distil-medium.en-decoder.int8.onnx \
  --tokens=./distil-medium.en-tokens.txt \
  --provider=cuda \
  ./0.wav \
  ./1.wav \
  ./8k.wav

/content/sherpa-onnx/sherpa-onnx/csrc/parse-options.cc:Read:361 sherpa-onnx-offline --whisper-encoder=./distil-medium.en-encoder.int8.onnx --whisper-decoder=./distil-medium.en-decoder.int8.onnx --tokens=./distil-medium.en-tokens.txt --provider=cuda ./0.wav ./1.wav ./8k.wav 

OfflineRecognizerConfig(feat_config=OfflineFeatureExtractorConfig(sampling_rate=16000, feature_dim=80), model_config=OfflineModelConfig(transducer=OfflineTransducerModelConfig(encoder_filename="", decoder_filename="", joiner_filename=""), paraformer=OfflineParaformerModelConfig(model=""), nemo_ctc=OfflineNemoEncDecCtcModelConfig(model=""), whisper=OfflineWhisperModelConfig(encoder="./distil-medium.en-encoder.int8.onnx", decoder="./distil-medium.en-decoder.int8.onnx", language="", task="transcribe"), tdnn=OfflineTdnnModelConfig(model=""), zipformer_ctc=OfflineZipformerCtcModelConfig(model=""), tokens="./distil-medium.en-tokens.txt", num_threads=2, debug=False, provider="cuda", model_type=""), lm_config=OfflineLMConf

# Run it with CPU

In [7]:
%%shell

export PATH=/content/sherpa-onnx/build/bin:$PATH

sherpa-onnx-offline \
  --whisper-encoder=./distil-medium.en-encoder.int8.onnx \
  --whisper-decoder=./distil-medium.en-decoder.int8.onnx \
  --tokens=./distil-medium.en-tokens.txt \
  --provider=cpu \
  ./0.wav \
  ./1.wav \
  ./8k.wav

/content/sherpa-onnx/sherpa-onnx/csrc/parse-options.cc:Read:361 sherpa-onnx-offline --whisper-encoder=./distil-medium.en-encoder.int8.onnx --whisper-decoder=./distil-medium.en-decoder.int8.onnx --tokens=./distil-medium.en-tokens.txt --provider=cpu ./0.wav ./1.wav ./8k.wav 

OfflineRecognizerConfig(feat_config=OfflineFeatureExtractorConfig(sampling_rate=16000, feature_dim=80), model_config=OfflineModelConfig(transducer=OfflineTransducerModelConfig(encoder_filename="", decoder_filename="", joiner_filename=""), paraformer=OfflineParaformerModelConfig(model=""), nemo_ctc=OfflineNemoEncDecCtcModelConfig(model=""), whisper=OfflineWhisperModelConfig(encoder="./distil-medium.en-encoder.int8.onnx", decoder="./distil-medium.en-decoder.int8.onnx", language="", task="transcribe"), tdnn=OfflineTdnnModelConfig(model=""), zipformer_ctc=OfflineZipformerCtcModelConfig(model=""), tokens="./distil-medium.en-tokens.txt", num_threads=2, debug=False, provider="cpu", model_type=""), lm_config=OfflineLMConfig